# PART 1 - READ IN THE DATAFRAMES

# Direct Investment Current Cost Adjustments

This is an excerpt of a longer script that calculates (A) the current cost markup to the direct investment equity position and (B) the current cost adjustment to direct investment equity income.

(A) involves revaluing the following from historical cost to current cost:
1. structures & equipment (fixed assets)
2. inventories
3. land

(B) involves computing the consumption of fixed capital:
1. economic depreciation at current cost for structures & equipment (fixed assets)


For A.1, industry-by-year net fixed assets (obtained using entity-level direct investment data from BEA's DI surveys) are revalued to current cost from historical cost using ratios derived from BEA's Fixed Assets Accounts. For B.1, depreciation is computed using depreciation rates implied by the Fixed Assets Accounts. For both, since the necessary values are published, the BEA API can be used to retrieve them (rather than downloading excel files on our website).

To install *beaapi*, visit https://github.com/us-bea/beaapi/releases and download the wheel (.whl) file. Then, open your Anaconda terminal and run the following code (if a new version has been released, update the file name):
```
    pip install Downloads\beaapi-0.0.2-py3-none-any.whl
```

You will also need an API key. To register, visit https://apps.bea.gov/api/signup/ and fill out the registration form on the right hand side of the page. The API key will be emailed to you.

## I. Preliminaries

Enter your BEA API key:

In [1]:
beakey = '12345678-2222-3333-4444-000000000099'

Not all years of available data from the Fixed Assets Accounts are used for these calculations. They begin with 1995, and end with the most recent year of data available on MNEs from the annual DI surveys. Below, the desired start and end years are assigned to variables:

In [2]:
yearfirst = 1995
yearlast = 2020


Import packages needed for this code:

In [3]:
import numpy as np
import pandas as pd
import beaapi

## II. Import Fixed Assets Data

The *beaapi* ```get_data_set_list``` method returns the names and descriptions of the available table series:

In [4]:
beaapi.get_data_set_list(beakey)

DatasetName                                 DatasetDescription
0                      NIPA                               Standard NIPA tables
1        NIUnderlyingDetail               Standard NI underlying detail tables
2                       MNE                          Multinational Enterprises
3               FixedAssets                       Standard Fixed Assets tables
4                       ITA                International Transactions Accounts
5                       IIP                  International Investment Position
6               InputOutput                                  Input-Output Data
7             IntlServTrade                       International Services Trade
8               IntlServSTA  International Services Supplied Through Affili...
9             GDPbyIndustry                                    GDP by Industry
10                 Regional                                 Regional data sets
11  UnderlyingGDPbyIndustry                         Underlying GDP by Industry
12       APIDatasetMetaData                  Metadata about other API datasets

The ```get_parameter_list``` method returns the parameters for which we will need to pass arguments when retrieving the data:

In [5]:
beaapi.get_parameter_list(beakey, 'FixedAssets')

ParameterName ParameterDataType  \
0     TableName            string   
1          Year           integer   

                              ParameterDescription  ParameterIsRequiredFlag  \
0                  The new Fixed Assets identifier                        1   
1  List of year(s) of data to retrieve (X for All)                        1   

  ParameterDefaultValue  MultipleAcceptedFlag AllValue  
0                                           0           
1                                           1        X

The first parameter is **TableName**. Tables are obtained from BEA API using their table names (IDs), which can be found [here](https://apps.bea.gov/api/_pdf/bea_web_service_api_user_guide.pdf). Fixed Assets table names follow the naming convention ```FAAt###X```. 

In [6]:
# A list of the table names
pftable_names = ['FAAt301E',
             'FAAt301S',
             'FAAt303E',
             'FAAt303S',
             'FAAt304E',
             'FAAt304S']
# same thing for residential FA tables:
rftable_names = ['FAAt501',
             'FAAt503',
             'FAAt504']

In [7]:
# The column names for each table
pfcol_names = ['ccposequip',
             'ccposstruct',
             'hcposequip',
             'hcposstruct',
             'ccdeprequip',
             'ccdeprstruct']
# same thing for residential FA tables:
rfcol_names = ['ccposres',
             'hcposres',
             'ccdeprres']


In [8]:
# Create 2 python dictionaires that map private FA table numbers to the descriptive col names that will be used later on to rename the "DataValue" col in each table:
# 
pfatables = dict(zip(pftable_names, pfcol_names))
# same thing for residential FA tables:
rfatables = dict(zip(rftable_names, rfcol_names))

In [9]:
# Check the dictionaries
pfatables, rfatables

({'FAAt301E': 'ccposequip',
  'FAAt301S': 'ccposstruct',
  'FAAt303E': 'hcposequip',
  'FAAt303S': 'hcposstruct',
  'FAAt304E': 'ccdeprequip',
  'FAAt304S': 'ccdeprstruct'},
 {'FAAt501': 'ccposres', 'FAAt503': 'hcposres', 'FAAt504': 'ccdeprres'})

**Year** is the other parameter. Use ```Year='X'``` as the argument for all years, ```Year='YYYY'``` for a single year, or ```Year='YYYY, YYYY, YYYY, ...'``` for multiple. Notice that, for multiple years, the argument is not a list of strings, but rather a flat string with commas between years. 

For many years it may be tedious to type them all manually, so you may wish to convert a list of numeric years to the flat string:

In [10]:
# create a list of numeric years using the variables defined in the previous section:
yearlist = [str(year) for year in range(yearfirst, yearlast+1)] 
yearstr = ', '.join(yearlist)

# Verify the string of years
print(type(yearstr), yearstr)

<class 'str'> 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020
